In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from pandas.tseries.offsets import MonthEnd

In [2]:
meta = pd.read_excel('ews_nb_1126.xlsx', sheet_name='meta', index_col='Data ID')

In [3]:
latest = pd.read_csv('latest_vintage_20251130.csv', index_col=0)
latest.index = pd.to_datetime(latest.index)
# 인덱스를 월말로 변경하고 빈도 설정
latest.index = latest.index + MonthEnd(0)
latest.index.freq = 'M'  # 월말 빈도 설정

In [4]:
latest

,gdp,reserve,short_ex_debt,bank_capital,bank_asset,bank_dep,bank_loan,sbank_capital,sbank_asset,nbank_rc,...,ins_mv,er_mv,gdp_growth_neg,cei_growth_neg,ipi_growth_neg,epu_pchg,epu_tp_pchg,esi_pchg_neg,nsi_pchg_neg,nonbank_gv
period,,,,,,,,,,,,,,,,,,,,,
1999-01-31,138829.0,53597833.0,35451.9,21264361.0,3.697396e+08,246309.9,201055.6,4511182.0,54009178.0,687839.8,...,0.137408,0.038965,-2.5,-0.003578,-0.002644,-0.099605,0.006888,0.000629,-0.000965,3.098334
1999-02-28,138829.0,55456770.0,35451.9,25192088.0,3.855952e+08,258259.9,201923.7,4511182.0,54009178.0,693467.2,...,0.145364,0.039643,-2.5,-0.008913,-0.002644,-0.038166,-0.021632,0.000629,-0.000965,2.773858
1999-03-31,138829.0,57450792.0,35451.9,21511258.0,4.273754e+08,270940.1,207814.0,4511182.0,54009178.0,677231.2,...,0.132367,0.034404,-2.5,-0.010247,-0.002644,-0.037349,0.002835,0.000629,-0.000965,2.551725
1999-04-30,148800.3,59263890.0,34655.9,26472932.0,4.141142e+08,269803.6,210659.2,4511182.0,54009178.0,691969.2,...,0.156702,0.034216,-3.1,-0.007695,-0.002644,-0.093268,0.244830,0.000629,-0.000965,2.718060
1999-05-31,148800.3,61409325.0,34655.9,27459447.0,4.150722e+08,267940.2,215663.7,4511182.0,54009178.0,699667.4,...,0.155300,0.033359,-3.1,-0.003818,-0.002644,-0.125977,-0.063806,0.000629,-0.000965,3.430014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-31,657836.9,411334711.0,161619.0,173394505.0,2.592687e+09,2136930.6,2453183.3,14298608.0,118794904.0,4032309.6,...,0.099543,0.016002,-0.7,-0.002018,0.001174,-0.038582,-0.120860,-0.007407,-0.032509,0.552131
2025-08-31,657836.9,416285737.0,161619.0,174763169.0,2.575903e+09,2162522.3,2467413.0,14305381.0,120690135.0,4071442.1,...,0.105257,0.017985,-0.7,-0.000000,-0.002350,-0.193278,-0.265383,-0.008088,-0.013035,0.331903
2025-09-30,657836.9,422015555.0,161619.0,176327003.0,2.654539e+09,2198983.7,2475267.1,14718925.0,123306904.0,4060028.4,...,0.098014,0.017146,-0.7,0.000671,-0.005569,-0.126399,-0.240591,-0.008753,-0.003512,0.094670


In [5]:
features = meta[meta['Feature']=='Y']

In [6]:
# Generate all mondays between 2002-01-01 and 2025-06-30
mondays = pd.date_range(start='2002-01-01', end='2025-11-24', freq='W-MON')

In [7]:
# Helper to get nth Monday of month
def get_nth_weekday_of_month(year, month, n, weekday=0):
    n = int(n)
    first_day = datetime(year, month, 1)
    days_offset = (weekday - first_day.weekday() + 7) % 7
    first_desired = first_day + timedelta(days=days_offset)
    return first_desired + timedelta(weeks=n-1)

In [8]:
output_dir = 'vintages'
os.makedirs(output_dir, exist_ok=True)

In [9]:
for today in mondays:
    # 이 빈티지에 포함할 과거 월말 전체 리스트
    all_periods = pd.period_range(latest.index[0], today.to_period('M'), freq='M')
    df = pd.DataFrame(index=all_periods.to_timestamp('M'), columns=features.index)
    
    for var in features.index:
        lag  = int(features.at[var, 'Delay'])
        week = int(features.at[var, 'Week'])
        
        for pe in all_periods:  # pe: Period('YYYY-MM')
            period_end = pe.to_timestamp('M')  # 월말 datetime
            if period_end not in latest.index:
                # latest에 월말 데이터가 없으면 무조건 NaN
                df.at[period_end, var] = np.nan
                continue
            
            if week == 0:
                # daily: 언제든 해당 월말 값 사용
                df.at[period_end, var] = latest.at[period_end, var]
            else:
                # monthly/quarterly: pub_month = pe + lag
                pub_month = pe + lag
                pub_date  = get_nth_weekday_of_month(pub_month.year, pub_month.month, week)
                # 빈티지 시점(today)이 pub_date 이후면 값 반영
                if today >= pd.Timestamp(pub_date):
                    df.at[period_end, var] = latest.at[period_end, var]
                else:
                    df.at[period_end, var] = np.nan
    
    # 파일로 저장 (ex: vintages/2008-04-07.csv)
    fname = os.path.join(output_dir, today.strftime('%Y-%m-%d') + '.csv')
    df.to_csv(fname)